In [5]:
!pip install yfinance

     ---------------------------------------- 0.0/939.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/939.5 kB ? eta -:--:--
     -- ------------------------------------ 61.4/939.5 kB 1.1 MB/s eta 0:00:01
     ----------- -------------------------- 286.7/939.5 kB 2.9 MB/s eta 0:00:01
     -------------------------- ----------- 655.4/939.5 kB 4.6 MB/s eta 0:00:01
     -------------------------------------- 939.5/939.5 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/101.1 kB ? eta -:--:--
   ---------------------------------------- 101.1/101.1 kB 6.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/112.

In [27]:
import yfinance as yf
import pandas as pd

def get_option_chain_data(instrument_name: str, expiry_date: str, side: str) -> pd.DataFrame:
    """Fetch option chain data for a given stock and expiry date."""
    stock = yf.Ticker(instrument_name)
    
    # Print available expiry dates for reference
    print(f"Available expiry dates for {instrument_name}: {stock.options}")
    
    # Check if the specified expiry date exists
    if expiry_date not in stock.options:
        raise ValueError(f"Expiry date {expiry_date} is not available for {instrument_name}.")

    # Validate the side input
    if side not in ['PE', 'CE']:
        raise ValueError("Side must be 'PE' for Puts or 'CE' for Calls.")

    # Get option chain data for the specified expiry date
    options_data = stock.option_chain(expiry_date)
    
    # Select either 'calls' or 'puts' based on the side
    option_type_data = options_data.puts if side == 'PE' else options_data.calls
    
    # Filter and prepare DataFrame
    option_chain = option_type_data[['strike', 'bid', 'ask']].copy()
    option_chain['instrument_name'] = instrument_name
    option_chain['expiry_date'] = expiry_date
    option_chain['side'] = side
    
    return option_chain.rename(columns={'strike': 'strike_price', 'bid': 'bid_price', 'ask': 'ask_price'})

# Example usage of Step 1
instrument_name = "AAPL"  # Example instrument
expiry_date = "2024-11-01"  # Example expiry date
side = "PE"  # 'PE' for Puts, 'CE' for Calls

try:
    df = get_option_chain_data(instrument_name, expiry_date, side)
    print(df)
except ValueError as e:
    print(e)


Available expiry dates for AAPL: ('2024-11-01', '2024-11-08', '2024-11-15', '2024-11-22', '2024-11-29', '2024-12-06', '2024-12-20', '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-17', '2025-06-20', '2025-08-15', '2025-09-19', '2025-12-19', '2026-01-16', '2026-06-18', '2026-12-18', '2027-01-15')
    strike_price  bid_price  ask_price instrument_name expiry_date side
0          100.0       0.00       0.01            AAPL  2024-11-01   PE
1          120.0       0.00       0.01            AAPL  2024-11-01   PE
2          130.0       0.00       0.01            AAPL  2024-11-01   PE
3          135.0       0.00       0.01            AAPL  2024-11-01   PE
4          140.0       0.00       0.04            AAPL  2024-11-01   PE
5          145.0       0.00       0.01            AAPL  2024-11-01   PE
6          150.0       0.00       0.01            AAPL  2024-11-01   PE
7          155.0       0.00       0.01            AAPL  2024-11-01   PE
8          160.0       0.00       0.01            AA

In [33]:
def get_margin_requirement(strike_price: float) -> float:
    """Calculate margin requirement based on strike price."""
    return strike_price * 0.1  # Assuming 10% of the strike price as the margin requirement

def calculate_margin_and_premium(data: pd.DataFrame, lot_size: int = 100) -> pd.DataFrame:
    """Calculate margin required and premium earned for options."""
    # Calculate margin and premium
    data['margin_required'] = data['strike_price'].apply(get_margin_requirement)
    data['premium_earned'] = data['bid_price'] * lot_size  # Using bid price for premium calculation

    # Select and rename final output columns
    final_output = data[['instrument_name', 'strike_price', 'side', 'bid_price', 'margin_required', 'premium_earned']]
    final_output = final_output.rename(columns={'bid_price': 'bid/ask'})  # Rename column for output
    
    return final_output

# Example usage for AAPL
instrument_name = "AAPL"  # Apple Inc.
side = "PE"  # 'PE' for Puts, 'CE' for Calls

try:
    # First, check available expiry dates
    stock = yf.Ticker(instrument_name)
    expiry_dates = stock.options
    print(f"Available expiry dates for {instrument_name}: {expiry_dates}")

    # Select an expiry date (change this based on what is available)
    expiry_date = expiry_dates[0] if expiry_dates else None  # Use the first available date

    if expiry_date:
        # Fetch option chain data
        df = get_option_chain_data(instrument_name, expiry_date, side)
        
        # Calculate margin and premium
        df_with_margin_and_premium = calculate_margin_and_premium(df)
        
        # Display the output
        print("\nOutput:")
        print(df_with_margin_and_premium.to_string(index=False))  # Print DataFrame without the index
    else:
        print("No expiry dates available for AAPL.")
except ValueError as e:
    print(e)


Available expiry dates for AAPL: ('2024-11-01', '2024-11-08', '2024-11-15', '2024-11-22', '2024-11-29', '2024-12-06', '2024-12-20', '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-17', '2025-06-20', '2025-08-15', '2025-09-19', '2025-12-19', '2026-01-16', '2026-06-18', '2026-12-18', '2027-01-15')
Available expiry dates for AAPL: ('2024-11-01', '2024-11-08', '2024-11-15', '2024-11-22', '2024-11-29', '2024-12-06', '2024-12-20', '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-17', '2025-06-20', '2025-08-15', '2025-09-19', '2025-12-19', '2026-01-16', '2026-06-18', '2026-12-18', '2027-01-15')

Output:
instrument_name  strike_price side  bid/ask  margin_required  premium_earned
           AAPL         100.0   PE     0.00            10.00             0.0
           AAPL         120.0   PE     0.00            12.00             0.0
           AAPL         130.0   PE     0.00            13.00             0.0
           AAPL         135.0   PE     0.00            13.50             0.0
      